<a href="https://colab.research.google.com/github/jagannath311/DataScience-/blob/master/language_sequence_labelling_using_ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import linecache


In [0]:
###it is a function to make a word into its constituent ngrams
def ngrams(n,word):
    list1=[]
    for i in range(len(word)):
            list1.append(word[i])
    for i in range(len(word)-1):
            list1.append(word[i]+word[i+1])
    for i in range(len(word)-2):
            list1.append(word[i]+word[i+1]+word[i+2])
    for i in range(len(word)-3):
            list1.append(word[i]+word[i+1]+word[i+2]+word[i+3])
    for i in range(len(word)-4):
            list1.append(word[i]+word[i+1]+word[i+2]+word[i+3]+word[i+4])
    return list1

In [0]:
##constructing a ngram file for each language
def ngramlan(x,y):
  t=open(x,'r')
  w=open(y,'w')

  for line in t:
    for word in line.split():
        l=[]
        l=ngrams(5,word)
        
        for i in l:
            w.write(i)
            w.write('\t')
        w.write('\n')
  t.close()
  w.close()

In [0]:
##already english ngrams are given making a dictionary out of it
t=open('enngrams.txt','r')
dic={}
enngrams={}
l=0
i=0
for line in t:
    for word in line.split():
        if l==0:
            if word not in dic:
                dic[word]=1
            else:
                dic[word]=dic[word]+1
    
        else:
            if len(word)==1:
                dic['label']='en'
                enngrams[i]=dic
                
                i=i+1
                dic={}
                dic[word]=1
                l=0
            else:
                l=0
                if word not in dic:
                    dic[word]=1
                else:
                    dic[word]=dic[word]+1
    l=l+1
    
    
t.close()
e=i

In [0]:
##training the each language classifier
def trainingBclassifier(a,b,c,fitted_models,labels):
  t=open(c,'r')
  i=e
  lanngrams={}
  f={}
  g=[]
  for line in t:
    dic={}
    for word in line.split():
      if word not in dic and word.isalpha():
          dic[word]=1
      elif word.isalpha():
          dic[word]=dic[word]+1
    dic['label']=labels
    
    lanngrams[i]=dic
    i=i+1
  t.close()
  
  f={**b,**lanngrams}
  df=pd.DataFrame.from_dict(f,orient='index')
  df.fillna(0,inplace=True)
  g=list(df.columns)
  y=df.label
  x=df.drop('label',axis=1)
  x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1234,stratify=df.label)
  pipelines={a:make_pipeline(StandardScaler(),GaussianNB()) }
  gnb_hyperparameters={'gaussiannb__priors':[None]}
  hyperparameters={a:gnb_hyperparameters}
  
  for name,pipeline in pipelines.items():
    model=GridSearchCV(pipeline,hyperparameters[name],cv=5)
    model.fit(x_train,y_train)
    fitted_models[name]=model
    print(name)
  return f,g

In [0]:
###driver code for the above functions
fitted_models={}
ngramlan('teluguw.txt','telugungrams.txt')
ngramlan('bengaliW.txt','bengalingrams.txt')
ngramlan('gujaratiW.txt','gujaratingrams.txt')
ngramlan('hindiW.txt','hindingrams.txt')
ngramlan('kannadaW.txt','kannadangrams.txt')
ngramlan('malayalamW.txt','malayalamngrams.txt')
ngramlan('maratiw.txt','maratingrams.txt')
ngramlan('tamil.txt','tamilngrams.txt')

te_en_ngrams,te_en_columns=trainingBclassifier('te_en_',enngrams,'telugungrams.txt',fitted_models,'te')
bn_en_ngrams,bn_en_columns=trainingBclassifier('bn_en_',enngrams,'bengalingrams.txt',fitted_models,'bn')
gu_en_ngrams,gu_en_columns=trainingBclassifier('gu_en_',enngrams,'gujaratingrams.txt',fitted_models,'gu')
hi_en_ngrams,hi_en_columns=trainingBclassifier('hi_en_',enngrams,'hindingrams.txt',fitted_models,'hi')
kn_en_ngrams,kn_en_columns=trainingBclassifier('kn_en_',enngrams,'kannadangrams.txt',fitted_models,'kn')
ml_en_ngrams,ml_en_columns=trainingBclassifier('ml_en_',enngrams,'malayalamngrams.txt',fitted_models,'ml')
ta_en_ngrams,ta_en_columns=trainingBclassifier('ta_en_',enngrams,'tamilngrams.txt',fitted_models,'ta')
mr_en_ngrams,mr_en_columns=trainingBclassifier('mr_en_',enngrams,'maratingrams.txt',fitted_models,'mr')

te_en_
bn_en_
gu_en_
hi_en_
kn_en_
ml_en_
ta_en_
mr_en_


In [0]:
##constructing ngrams at the sentence level
def sentngrams(a,b,c,d):
  t=open(a,'r')
  w=open(b,'r')
  z=open(c,'w')
  y=open(d,'w')
  langu=['en','te','kn','ta','gu','bn','hi','mr','ml']
  engu=['en']
  l=[]
  for line in t:
    list1=[]
    l=list(w.readline().split())
    list1=list(set(l)&set(langu))
    if (len(list1)==2 and 'en' in list1) or list1==engu:
      list2=[]
      list2=list1
      for i in list2:
        z.write(i)
        z.write("_")
      for word in line.split():
        if word.isalpha():
          list2=[]
          list2=ngrams(5,word)
          for i in list2:
            z.write('\t')
            z.write(i)
      for word in l:
        y.write(word)
        y.write('\t')
    y.write('\n')    
    z.write('\n')
  t.close()
  w.close()
  z.close()
  y.close()

In [0]:
##drivercode for above function
sentngrams('InputTraining.txt','AnnotationTraining.txt','ngramstrain.txt','ngramstrainannote.txt')
sentngrams('InputTesting.txt','AnnotationTesting.txt','ngramstest.txt','ngramstestannote.txt')
##making the sentence level ngrams into a dataframe for constructing classifier
t=open('ngramstrain.txt','r')
i=0
ngrams1={}
for line in t:
    dic={}
    l=list(line.split())
    if len(l)>0:
      dic['label']=l[0]
      for j in range(1,len(l)):
        if l[j] not in dic and l[j].isalpha():
            dic[l[j]]=1
        elif l[j].isalpha():
            dic[l[j]]=dic[l[j]]+1
    
    
    ngrams1[i]=dic
    i=i+1
t.close()

In [0]:
##constrcuting the classifiers
df=pd.DataFrame.from_dict(ngrams1,orient='index')
df.fillna(0,inplace=True)
df.label.replace('en_bn_','bn_en_',inplace=True)
df.label.replace('en_kn_','kn_en_',inplace=True)
df.label.replace('en_te_','te_en_',inplace=True)
df.label.replace('en_mr_','mr_en_',inplace=True)
df.label.replace('en_ml_','ml_en_',inplace=True)
df.label.replace('en_gu_','gu_en_',inplace=True)
df.label.replace('en_ta_','ta_en_',inplace=True)
df.label.replace('en_hi_','hi_en_',inplace=True)
y=df.label
x=df.drop('label',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=1234)
pipelines1={
    
    'gnb':make_pipeline(StandardScaler(),GaussianNB()),
    'mlp':make_pipeline(StandardScaler(),MLPClassifier(random_state=123)),
    'l2':make_pipeline(StandardScaler(),LogisticRegression(penalty='l2',random_state=123))
}
gnb_hyperparameters={'gaussiannb__priors':[None]}
mlp_hyperparameters={'mlpclassifier__activation':['relu'],'mlpclassifier__max_iter':[200]}
l2_hyperparameters={'logisticregression__max_iter':[1000]}
hyperparameters={'gnb':gnb_hyperparameters,'mlp':mlp_hyperparameters,'l2':l2_hyperparameters,}
fitted_models1={}
for name,pipeline in pipelines1.items():
    model=GridSearchCV(pipeline,hyperparameters[name],cv=5)
    model.fit(x_train,y_train)
    fitted_models1[name]=model
    print(name)



gnb
mlp
l2


In [0]:
##predict at word level
def predict(pred1,file):
  preda=[]
  t=open(file,'w')  
  j=0
  lane=0
  for i in pred1:
    word_list=[]
    k=0
    d=x_test.index.values[j]
    line = linecache.getline('ngramstrain.txt',d+1)
    line1=linecache.getline('ngramstrainannote.txt',d+1)
    str2=''
    for word in line.split():
      if len(word)==1:
        str2=str2+word
      elif len(str2)>0:
        word_list.append(str2)
        str2=''
    if i!='en_':
     
      list2=[]
     
      
      for j in range(len(word_list)):
        dic={}
        we=col[i]
        for some in we:
          dic[some]=0
        list1=[]
        list1=ngrams(5,word[j])
        for word1 in list1:
          if word1 in dic:
            dic[word1]=dic[word1]+1
        df=pd.DataFrame.from_dict(dic,orient='index')
        df=df.transpose()
        df=df.drop('label',axis=1)
        df.append(dic,ignore_index=True)
        pred=fitted_models[i].predict(df)
        t.write('\n')
        t.write(word[j])
        t.write('\t')
        t.write(line1[j])
        t.write('\t')
        t.write(pred[0])
        
        preda.append(pred)
    else:
      ##all will be english
      for word in word_list:
        t.write('\n')
        t.write(word)
        t.write('\t')
        t.write('en')
        preda.append('en')

    j=j+1
    print(j)
    t.write('\n')
  t.close()  
  return preda

In [0]:
##driver code for prediction and savving into a text file
o={'te_en_':te_en_ngrams,'bn_en_':bn_en_ngrams,'kn_en_':kn_en_ngrams,'hi_en_':hi_en_ngrams,'gu_en':gu_en_ngrams,'ml_en_':ml_en_ngrams,'mr_en_':mr_en_ngrams,
   'ta_en_':ta_en_ngrams
}
col={'te_en_':te_en_columns,'bn_en_':bn_en_columns,'kn_en_':kn_en_columns,'hi_en_':hi_en_columns,'gu_en_':gu_en_columns,'ml_en_':ml_en_columns,'mr_en_':mr_en_columns,
   'ta_en_':ta_en_columns}


for name,model in fitted_models1.items():
  pred=model.predict(x_test)
  if name=='gnb':
    preda=predict(pred,'predicted_words_gnb.txt')
  elif name=='mlp':
    preda=predict(pred,'predicted_words_mlp.txt')
  elif name=='l2':
     preda=predict(pred,'predicted_words_l2.txt')
  print(f1_score(y_test,pred,average='weighted'))